In [1]:
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn

# Inception
---
既能保持网络结构的稀疏性，又能利用密集矩阵的高计算性能

In [2]:
class Inception(nn.Block): # Inception 的通道数分配之比是ImageNet数据集上大量实验得来的！！
    # c1 - c4 为每条线路里的层的输出通道数
    def __init__(self,c1,c2,c3,c4,**kwargs):
        super(Inception,self).__init__(**kwargs)
        
        # 线路1，单1*1卷积层
        self.p1_1 = nn.Conv2D(c1,kernel_size=1,activation='relu')
        
        # 线路2，1*1 + 3*3 卷积层
        self.p2_1 = nn.Conv2D(c2[0],kernel_size=1,activation='relu')
        self.p2_2 = nn.Conv2D(c2[1],kernel_size=3,padding=1,activation='relu')
        
        # 线路3，1*1 + 5*5 卷积层
        self.p3_1 = nn.Conv2D(c3[0],kernel_size=1,activation='relu')
        self.p3_2 = nn.Conv2D(c3[1],kernel_size=5,padding=2,activation='relu')
        
        # 线路4，3*3 + 1*1 卷积层
        self.p4_1 = nn.MaxPool2D(pool_size=3,strides=1,padding=1)
        self.p4_2 = nn.Conv2D(c4,kernel_size=1,activation='relu')
        
    def forward(self,x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return nd.concat(p1,p2,p3,p4,dim=1) # 在通道维上连结输出

# 主体
---
- 主体卷积部分使用5个block
    1. 每个block间使用3\*3 maxPooling 来减小输出高宽。

In [3]:
# 模块1
b1 = nn.Sequential() # 使用64通道的 7*7卷积层
b1.add(nn.Conv2D(64,kernel_size=7,strides=2,padding=3,activation='relu'), # output : 64*32*32
      nn.MaxPool2D(pool_size=3,strides=2,padding=1)) # ouput : 64*16*16

In [4]:
# 模块2
b2 = nn.Sequential() # 64 通道的 1*1 卷积层
b2.add(nn.Conv2D(channels=64,kernel_size=1,activation='relu'),
      nn.Conv2D(channels=192,kernel_size=3,padding=1,activation='relu'), # 将通道增大了3倍，对应Inception中的第二条线路
      nn.MaxPool2D(pool_size=3,strides=2,padding=1)) 

In [6]:
# 模块3
b3 = nn.Sequential()
b3.add( # 192 是上方的输入channel!!
    # Inception1 输出通道数：64 + 128 + 32 + 32 = 256（2:4:1:1） ， 
    # 其中 2 & 3 线路将输入通道减少至 96/192 = 1/2 & 16/192 = 1/12
      Inception(64,(96,128),(16,32),32),
    
    # Inception2 输出通道数：128 + 192 + 96 + 64 = 480 （4:6:3:2）， 
    # 其中 2 & 3 线路将输入通道减少至128/256 = 1/2 & 32/256 = 1/8 
      Inception(128,(128,192),(32,96),64),
    
      nn.MaxPool2D(pool_size=3,strides=2,padding=1))

In [7]:
# 模块4
b4 = nn.Sequential()
b4.add(
    # Inception1 输出通道数:
    Inception(192,(96,208),(16,48),64),
    Inception(160,(112,224),(24,64),64),
    Inception(128,(128,256),(24,64),64), # 前三个输出通道数都是512
    Inception(112,(144,288),(32,64),64), # 528
    Inception(256,(160,320),(32,128),128), # 832
    
    nn.MaxPool2D(pool_size=3,strides=2,padding=1)
    )

In [8]:
b5 = nn.Sequential()
b5.add(
        Inception(256,(160,320),(32,128),128), # total channel : 832
        Inception(384,(192,384),(48,128),128), # total channel : 1024
        
        # 和NiN一样，使用全局平均池化层，将每个通道宽高变成1 
        nn.GlobalAvgPool2D()
)

In [9]:
net = nn.Sequential()
net.add(b1,b2,b3,b4,b5,nn.Dense(10)) # 注意后方，十个分类！！个数为标签类别

In [11]:
# 这里测试各模块间的输出变化
X = nd.random.uniform(shape=(1,1,96,96))
net.initialize()
for layer in net:
    y_hat = layer(X)
    print(layer.name,"output shape:\t",y_hat.shape)

sequential0 output shape:	 (1, 64, 24, 24)
sequential1 output shape:	 (1, 192, 48, 48)
sequential3 output shape:	 (1, 480, 48, 48)
sequential4 output shape:	 (1, 832, 48, 48)
sequential5 output shape:	 (1, 1024, 1, 1)
dense0 output shape:	 (1, 10)


# 训练模型
---

In [ ]:
lr,num_epochs,batch_size,ctx = 0.1,5,128,d2l.try_gpu()
net.initialize(force_reinit=True,ctx=ctx,init = init.Xavier())
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)
